# Pipeline
In diesem File findet die ganze Verarbeitung der Daten statt. Die Einstellungen sind im unteren Codeblock zu finden.

Hier gäbe es performancetechnisch noch einiges zu optimieren, jedoch liegt das ausserhalb des Scopes dieser Arbeit.

# Preparing
Vorbereitung der Packages und Einstellungen

In [1]:
import requests
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

# Settings
path_tickets_export = "Daten/tickets_export.xlsx"
stopwords_url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-de/master/stopwords-de.txt'
num_top_words_per_tickets = 10
num_top_words_total = 50

# Preprocessing
Datenverarbeitung als Grundlage für die nachfolgenden Schritte

In [2]:
data = pd.read_excel(path_tickets_export)

# Process Dates
data['Erstellt'] = pd.to_datetime(data['Erstellt'], unit="D", origin=pd.Timestamp('1899-12-30')).dt.strftime('%Y-%m-%d %H:%M:%S')
data['Aktualisiert'] = pd.to_datetime(data['Aktualisiert'], unit="D", origin=pd.Timestamp('1899-12-30')).dt.strftime('%Y-%m-%d %H:%M:%S')
data['Gelöst'] = pd.to_datetime(data['Gelöst'], unit="D", origin=pd.Timestamp('1899-12-30')).dt.strftime('%Y-%m-%d %H:%M:%S')

data[["Erstellt", "Aktualisiert", "Gelöst"]]

,Erstellt,Aktualisiert,Gelöst
0,2022-07-31 12:59:24,2022-08-14 00:00:01,2022-07-31 16:37:53
1,2022-07-31 10:22:23,2022-08-10 00:01:27,2022-08-02 06:59:32
2,2022-07-31 05:25:30,2022-08-23 00:00:41,2022-08-15 15:34:45
3,2022-07-30 22:59:57,2022-08-12 13:34:32,2022-08-08 09:04:23
4,2022-07-30 21:53:21,2022-08-10 00:01:27,2022-08-02 07:59:14
...,...,...,...
63507,2022-12-01 07:06:13,2022-12-07 00:00:06,2022-12-01 08:12:54
63508,2022-12-01 07:06:10,2022-12-01 07:06:10,2022-12-01 07:06:10
63509,2022-12-01 07:05:35,2022-12-08 09:00:19,2022-12-01 08:13:12
63510,2022-12-01 07:04:47,2022-12-08 10:00:15,2022-12-01 09:45:00


In [3]:
# Process Text
data['Betreff'] = data['Betreff'].astype(str).copy()
data['Beschreibung'] = data['Beschreibung'].astype(str).copy()
data['Auflösungshinweise'] = data['Auflösungshinweise'].astype(str).copy()

def process_captions(data, column):
    data[column] = data[column].apply(lambda x: x.replace('\n', ' '))
    data[column] = data[column].apply(lambda x: x.replace('--', ' '))
    data[column] = data[column].apply(lambda x: x.replace(' - ', ' '))
    data[column] = data[column].apply(lambda x: x.replace(':', ' '))
    data[column] = data[column].apply(lambda x: x.replace(';', ' '))
    data[column] = data[column].apply(lambda x: x.replace(',', ' '))
    data[column] = data[column].apply(lambda x: x.replace('.', ' '))
    data[column] = data[column].apply(lambda x: x.replace('!', ' '))
    data[column] = data[column].apply(lambda x: x.replace('?', ' '))
    data[column] = data[column].apply(lambda x: x.replace('(', ' '))
    data[column] = data[column].apply(lambda x: x.replace(')', ' '))
    data[column] = data[column].apply(lambda x: x.lower())
    data[column] = data[column].apply(lambda x: x.replace('[^A-Za-z]', ''))
    return data[column]

data['Betreff'] = process_captions(data, 'Betreff').copy()
data['Beschreibung'] = process_captions(data, 'Beschreibung').copy()
data['Auflösungshinweise'] = process_captions(data, 'Auflösungshinweise').copy()

data[["Betreff", "Beschreibung", "Auflösungshinweise"]].head()

,Betreff,Beschreibung,Auflösungshinweise
0,tribuna funktioniert nicht kommt ständig eine ...,siehe oben category software ...,file system des datenbank systems ist vollgela...
1,passwort zurücksetzen für saxer vera gsi-zshk...,guten tag ich habe untenstehende mail erhalte...,guten morgen frau saxer besten dank für die i...
2,merci d'ouvrir les accès intranet ajv-rgmo à t...,les accès intranet ajv-rgmo à tamara tedde mm...,dieses ticket gibt es in doppelter ausführung ...
3,kein zugriff mehr auf den hauptposteingang und...,category software subcategor...,der fehler konnte behoben werden
4,personne détenue en doublon,la personne détenue n'a été entrée qu'une seul...,guten tag die id person 201691 ist nur einmal...


# TF-IDF
Berechnung der Keywords mittels TF-IDF

In [4]:
# Concat Free Text
data["freetext"] = data["Betreff"] + " " + data["Beschreibung"] + " " + data["Auflösungshinweise"]

# Fit TF-IDF with Stopwords
stopwords = requests.get(stopwords_url).text.split("\n")
tfidf = TfidfVectorizer(stop_words=stopwords)
tfidf.fit(data["freetext"])

# Convert vocabulary to DataFrame
df_tfidf = pd.DataFrame.from_dict(tfidf.vocabulary_, orient="index")
df_tfidf.sort_values(by=0, ascending=False)
df_tfidf = df_tfidf.reset_index()
df_tfidf.columns = ["word", "index"]

# Get Top Words
top_words_i_list = []
for i, row in tqdm(data.iterrows()):
    top_words = sorted(
        list(enumerate(tfidf.transform([row["freetext"]]).toarray()[0])),
        key=lambda x: x[1],
        reverse=True,
    )[:num_top_words_per_tickets]

    # create empty list to store top words 
    top_words_i = []
    for word_index, word_score in top_words:
        top_words_i.append(df_tfidf[df_tfidf["index"] == word_index]["word"].values[0])
    
    # append top words to top_words_i_list
    top_words_i_list.append(top_words_i)

# Add top words to data
data["top_words_each_ticket"] = top_words_i_list

63512it [16:40, 63.46it/s]


In [5]:
# Export to Excel and Parquet
data.to_excel("Daten/tickets_export_topn_words.xlsx", index=False)
print("Exported to Excel")
data.to_parquet("Daten/tickets_export_topn_words.parquet", index=False)
print("Exported to Parquet")

Exported to Excel
Exported to Parquet


# OneHotEncoding Top N Words
OneHotEncodierung der wichtigsten Wörter im ganzen Datensatz

In [6]:
data = pd.read_parquet("Daten/tickets_export_topn_words.parquet")

# get all top words from all tickets
important_words_df = pd.DataFrame(data["top_words_each_ticket"].explode()).value_counts()

# convert from multiindex to single index
important_words_df = important_words_df.reset_index().set_index("top_words_each_ticket")

# filter words
## remove words with numbers 
important_words_df = important_words_df[important_words_df.index.str.contains(r'[0-9]') == False]

## if none remove
important_words_df = important_words_df[important_words_df.index != "none"]
important_words_df = important_words_df[important_words_df.index != "nan"]

# get top num_top_words_total words
important_words = important_words_df.head(num_top_words_total)

# onehot encoding important words
for word in tqdm(important_words.index):
    data[word] = data["top_words_each_ticket"].apply(lambda x: 1 if word in x else 0)

100%|██████████| 50/50 [00:07<00:00,  7.07it/s]


In [7]:
# Export to Excel and Parquet
data.to_excel("Daten/tickets_export_topn_words_onehotencoded.xlsx", index=False)
print("Exported to Excel")
data.to_parquet("Daten/tickets_export_topn_words_onehotencoded.parquet", index=False)
print("Exported to Parquet")

Exported to Excel
Exported to Parquet


# Get coordinates 
Koordinaten zu den Orten hinzufügen

In [9]:
data = pd.read_parquet("Daten/tickets_export_topn_words_onehotencoded.parquet")

# function to get the coordinates of a location
def get_coordinates(location):
    url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
    response = requests.get(url)
    data = response.json()
    return (data[0]['lon'], data[0]['lat']) if len(data) > 0 else (None, None)

# get all the unique locations and their coordinates
unique_locations = data["Ort"].unique()
unique_coordinates = [get_coordinates(location) for location in tqdm(unique_locations)]

# create a dataframe with the locations and coordinates
streetmapping = pd.DataFrame({"location": unique_locations, "coordinates": unique_coordinates})

# get streets with no coordinates
missing_streets = streetmapping[streetmapping["coordinates"] == (None, None)].copy()

# replace \n and other stuff with a comma
missing_streets["location"] = missing_streets["location"].str.replace("\n", ", ").replace("\r", ", ").replace("\t", ", ").copy()

# remove middlepart of the location
# e.g. "Bernstrasse 5, Postfach 207, 3312 Fraubrunnen" -> "Bernstrasse 5, 3312 Fraubrunnen"
missing_streets["location"] = missing_streets["location"].str.split(", ").str[0] + ", " + missing_streets["location"].str.split(", ").str[-1].copy()

# get the coordinates for the missing streets
missing_coordinates = [get_coordinates(location) for location in tqdm(missing_streets["location"])]

# add the coordinates to the dataframe
missing_streets["coordinates"] = missing_coordinates

# add the missing coordinates to the streetmapping dataframe
streetmapping.loc[streetmapping["coordinates"] == (None, None), "coordinates"] = missing_streets["coordinates"]

# get the missing streets once again
missing_streets = streetmapping[streetmapping["coordinates"] == (None, None)].copy()

# get last part of the location
# e.g. "Bernstrasse 5, Postfach 207, 3312 Fraubrunnen" -> "3312 Fraubrunnen"
missing_streets["location"] = missing_streets["location"].str.split(", ").str[-1:].str.join(", ").copy()
# add ", Switzerland" to the location
missing_streets["location"] = missing_streets["location"] + ", Switzerland"


# get the coordinates for the missing streets
missing_coordinates = [get_coordinates(location) for location in tqdm(missing_streets["location"])]

# add the coordinates to the dataframe
missing_streets["coordinates"] = missing_coordinates

# add the missing coordinates to the streetmapping dataframe
streetmapping.loc[streetmapping["coordinates"] == (None, None), "coordinates"] = missing_streets["coordinates"]

# convert coordinates to lon and lat columns
streetmapping["lon"] = streetmapping["coordinates"].str[0].astype(float)
streetmapping["lat"] = streetmapping["coordinates"].str[1].astype(float)

# remove the coordinates column
streetmapping = streetmapping.drop("coordinates", axis=1)

100%|██████████| 81/81 [00:41<00:00,  1.94it/s]


In [10]:
# Export to Excel and Parquet
streetmapping.to_excel("Daten/streetmapping.xlsx", index=False)
print("Exported to Excel")
streetmapping.to_parquet("Daten/streetmapping.parquet", index=False)
print("Exported to Parquet")

Exported to Excel
Exported to Parquet


In [11]:
# Cleanup data
data = data.merge(streetmapping, left_on='Ort', right_on='location').copy()
data = data.drop(columns=['location']).copy()

In [12]:
# Export to Excel and Parquet
data.to_excel("Daten/tickets_export_topn_words_onehotencoded_with_coordinates.xlsx", index=False)
print("Exported to Excel")
data.to_parquet("Daten/tickets_export_topn_words_onehotencoded_with_coordinates.parquet", index=False)
print("Exported to Parquet")

Exported to Excel
Exported to Parquet
